<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Visualizing Survey Property Maps</b> <br>
Contact author(s): Eli Rykoff <br>
Last verified to run: 2022-08-10 <br>
LSST Science Piplines version: Weekly 2022_22 <br>
Container Size: large <br>
Targeted learning level: intermediate <br>

**Description:** Demonstrate tools to visualize survey property maps.

**Skills:** Load and visualize survey property maps using healsparse and skyproj.

**LSST Data Products:** Survey property maps.

**Packages:** healsparse, skyproj, lsst.daf.butler

**Credit:**
This notebook was originally developed by Eli Rykoff with editing from Alex Drlica-Wagner and Melissa Graham.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

This notebook will teach the user how to load and visualize survey property maps generated by the Rubin Science Pipelines. Data products are accessed through the Butler, and the user is expected to be familiar with the content of the introductory Butler tutorial. It introduces two new packages, `healsparse` and `skyproj`, for working with survey property maps.

### 1.1 Package Imports

Import general python packages and the Butler from the science pipelines.

Import two additional packages for working with the survey property maps.

The `healsparse` packages provides utilities for reading and manipulating sparse healpix maps.
More information can be found in the documentation "[HealSparse: A sparse implementation of HEALPix](https://healsparse.readthedocs.io/en/latest/)".

The `skyproj` package provides utilities for visualizing both sparse and dense HEALPix maps, as described in the documentation "[SkyProj: Sky Projections with matplotlib and PROJ](https://skyproj.readthedocs.io/en/latest/)".

> **Notice:** The following cell will output a warning about a version mismatch between CFITSIO and a linked library when used with version Weekly 2022_22. This warning can be safely ignored and will be corrected in the future.

In [ ]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval, LinearStretch, ImageNormalize

# packages for working with sparse healpix maps
import healsparse as hsp
import skyproj

# LSST packages
from lsst.daf.butler import Butler
import lsst.geom as geom

# allow interactive plots
%matplotlib widget

# default plot style is accessible
plt.style.use('tableau-colorblind10')

## 2. Access Survey Property Maps

Survey property maps are created as part of the LSST Science Pipelines.
They take the form of sparse HEALPix maps, where the survey property at each spatial pixel is identified by a pixel number/pixel value pair.

Start by creating an instance of the Butler and using it to access these maps for DP0.

In [ ]:
config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = Butler(config, collections=collections)

Determine which property maps are available for the survey.

> **Notice:** The following cell may produce a warning about the 'ScarletModelData' storage class, which is safe to ignore.

In [ ]:
# Display the available dataset types
registry=butler.registry
for d in sorted(registry.queryDatasetTypes()): print(d.name,".....\t",d.nameAndComponent)

In [ ]:
for dtype in sorted(butler.registry.queryDatasetTypes(expression="*consolidated_map*")):
    print(dtype.name)

Each of these products represents a healsparse map containing the value of an individual survey property.

The meaning of these types is:

* `deepCoadd_dcr_ddec_consolidated_map_weighted_mean`: Average effect of differential chromatic refraction (DCR) in declination direction
* `deepCoadd_dcr_dra_consolidated_map_weighted_mean`: Average effect of differential chromatic refraction (DCR) in right ascension direction
* `deepCoadd_dcr_e1_consolidated_map_weighted_mean`: Average effect of differential chromatic refraction (DCR) on psf e1
* `deepCoadd_dcr_e2_consolidated_map_weighted_mean`: Average effect of differential chromatic refraction (DCR) on psf e2
* `deepCoadd_exposure_time_consolidated_map_sum`: Total exposure time (seconds)
* `deepCoadd_psf_e1_consolidated_map_weighted_mean`: Weighted mean of psf e1 of images input to coadd
* `deepCoadd_psf_e2_consolidated_map_weighted_mean`: Weighted mean of psf e2 of images input to coadd
* `deepCoadd_psf_maglim_consolidated_map_weighted_mean`: PSF Flux 5-sigma magnitude limit (AB)
* `deepCoadd_psf_size_consolidated_map_weighted_mean`: Weighted mean of psf size of images input to coadd (pixels)
* `deepCoadd_sky_background_consolidated_map_weighted_mean`: Weighted mean of sky background of images input to coadd (ADU)
* `deepCoadd_sky_noise_consolidated_map_weighted_mean`: Weighted mean of sky noise of images input to coadd (ADU)


Note that the DCR maps are proportionality maps; that is, the expected effect will be proportional to the value in the map with an arbitrary/empirically derived constant of proportionality.

Read a map by specifying the map name and a band.

In [ ]:
# No consolidated map
all_map_names = []
for dtype in sorted(butler.registry.queryDatasetTypes(expression="*map*")):
    if "deepCoadd" in dtype.name and "consolidated_map" in  dtype.name :
        all_map_names.append(dtype.name)

In [ ]:
all_map_names

In [ ]:
index=4

In [ ]:
selected_map_name = all_map_names[index]

In [ ]:
all_hspmaps = []
all_bands = ['u','g','r','i','z','y']
for band in all_bands:
    hspmap = butler.get(selected_map_name, band=band)
    all_hspmaps.append(hspmap)

In [ ]:
len(all_hspmaps)

## 3. Manipulating Survey Property Maps

The survey property maps are provided as HealSparseMap objects.

We provide a few very brief examples here.

To conserve memory, HealSparse uses a dual-map approach, where a low-resolution full-sky “coverage map” is combined with a high resolution map containing the pixel data where it is available. It is easy to find the resolution of these maps.

In [ ]:
print(hspmap)

Each pixel of the healsparse map corresponds to a small region of the sky.
The value of the map corresponds to the value of the survey property at that location.

To access the survey property value at a specific location or set of locations, query for the map value using the `get_values_pos` functionality.

In [ ]:
print(hspmap.get_values_pos(60, -37))

Query for the map value at an array of locations.

In [ ]:
ra = np.linspace(59.5, 60.5, 5)
print('RA: ', ra)
dec = np.linspace(-37.5, -36.5, 5)
print('Dec: ', dec)
for d in dec:
    print(hspmap.get_values_pos(ra, d))
del ra, dec

If you ask for the value of the map outside of the region where it is defined, you will get a sentinel value.

In [ ]:
print(hspmap.get_values_pos(180, 0))

## 4. Visualizing Survey Property Maps

Now that we know how to access the values of the healsparse map, we can put together our own simple visualization by creating a grid of RA, Dec values and asking for the map values.
We can then plot these values with matplotlib.
Note that if you pan or zoom, the map does not update in resolution or coverage area.

In [ ]:
ix=0
iy=0
for idx in range(6):
    iy=idx//3
    ix=idx-iy*3
   
   
    print("ix",ix,'iy',iy)

In [ ]:
def PlotSurveyPropertyMaps(maps,filters,title):

    ra = np.linspace(59.5, 60.5, 100)
    dec = np.linspace(-37.5, -36.5, 100)
    x, y = np.meshgrid(ra, dec)
    
    
    fig, ax = plt.subplots(2, 3, figsize=(12, 8))
    
    idx=0
    ix=0
    iy=0
    
    for themap in maps:
   
        iy=idx//3
        ix=idx-iy*3
    
        print("idx",idx,"ix",ix,'iy',iy, "filter = ",filters[idx])
        
        values = maps[idx].get_values_pos(x, y)

        
        
        ax[iy,ix].pcolormesh(x, y, values) 

        ax[iy,ix].set_xlabel("Right Ascension (deg)")
        ax[iy,ix].set_ylabel("Declination (deg)")
        label=" band " + filters[idx]
        ax[iy,ix].set_title(label)
        ax[iy,ix].axis('tight')
        #ax[ix,iy].colorbar(label=title)

        del values
        idx+=1

    plt.tight_layout()
    plt.suptitle(title,y=1.003)
    plt.show()
    del fig, ra, dec, x, y
    del ax

In [ ]:
PlotSurveyPropertyMaps(all_hspmaps ,all_bands,selected_map_name)

The `skyproj` package provides much more advanced plotting capabilities.
Here we will demonstrate some basic use-cases for dealing with survey property maps.

The use of `McBrydeSkyproj` with `lon_0=65.0` creates a visualization using the McBryde-Thomas Flat Polar Quartic projection, centered at 65 deg longitude, which is appropriate for the DP0.2 footprint.

In [ ]:
filt_idx = 5
the_hspmap = all_hspmaps[filt_idx] 

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(the_hspmap)
sp.draw_colorbar(label=selected_map_name)
thetitle = selected_map_name + " band " + all_bands[filt_idx]
plt.suptitle(thetitle,y=1.001)
plt.show()

del fig, ax, sp